In [91]:
import re
# keyword = '예방 or 치료 adj 질병 or 진단 NEAR2 방어 or 치료 and 진단 키트 or 뭥미 and 진단 NEAR0 방어'
# keyword = '예방 and 치료 and 질병 and (진단 키트)'
# keyword = '예방 치료 질병 (진단 키트)'
# keyword = '예방 or 치료 adj 질병 or 진단 NEAR2 방어 or 치료 and 진단 키트 or 뭥미 and 진단 NEAR0 방어'
# keyword = '예방 and 치료 and 질병 and (진단 near 키트)'
# keyword = '예방 and 치료 and 진단 and (진단 adj 키트 or 조성물) and (진단 adj1 키트) and 진단 adj2 키트 and 진단 adj0 키트'
# (예방&치료&진단&(진단<->키트|조성물)&(진단<1>키트)&(진단<2>키트)&진단<->키트)
# select count(*) over () as cnt, 등록사항, 발명의명칭, 출원번호, 출원일, 출원인1, 출원인코드1, 출원인국가코드1, 발명자1, 발명자국가코드1, 등록일, 공개일, ipc코드, 요약, 청구항 FROM kr_text_view WHERE ((("search" @@ to_tsquery('((예방)&(치료)&(진단)&(진단<->키트)|(전염))')))) order by  출원일 DESC
# (예방&치료&진단&(진단<->키트|조성물)&(진단<1>키트)&(진단<2>키트)&진단<->키트)
# select count(*) over () as cnt, 등록사항, 발명의명칭, 출원번호, 출원일, 출원인1, 출원인코드1, 출원인국가코드1, 발명자1, 발명자국가코드1, 등록일, 공개일, ipc코드, 요약, 청구항 FROM kr_text_view WHERE ((("search" @@ to_tsquery('((예방)&(치료)&(진단)&(진단<->키트)|(전염))')))) order by  출원일 DESC
keyword = '(예방) and (치료) and (진단) and (진단 near 키트 or 백신 조성물)'
# keyword = '(진단 near 키트 or 대장)'
print(keyword)

def tsquery_keywords(keyword="", fieldName="", mode="terms"):

    if not keyword:
        return None    

    adjHaveOnlyZero='( adj[0]\d* )'
    adjOnly = '( adj )'
    adjZeroGroup = r'|'.join((adjHaveOnlyZero,adjOnly))
    adjHaveNumberExecptZero=r' adj([1-9]\d*) '
    adjSpace = r'(\([-!:*ㄱ-ㅎ|가-힣|a-z|A-Z|0-9]+) ([-!:*ㄱ-ㅎ|가-힣|a-z|A-Z|0-9]+\))'

    nearHaveNumberExecptZero = '([-!:*|ㄱ-ㅎ|가-힣|a-z|A-Z|0-9]+) near([1-9]\d*) ([-!:*|ㄱ-ㅎ|가-힣|a-z|A-Z|0-9]+)'
    nearHaveOnlyZero='([-!:*|ㄱ-ㅎ|가-힣|a-z|A-Z|0-9]+) near[0]\d* ([-!:*|ㄱ-ㅎ|가-힣|a-z|A-Z|0-9]+)'
    nearOnly='([-!:*|ㄱ-ㅎ|가-힣|a-z|A-Z|0-9]+) near ([-!:*|ㄱ-ㅎ|가-힣|a-z|A-Z|0-9]+)' 

    removeDate = r'( and \(\@AD.*\d{8}\))'
    removeAP = r'( and \([ -!:*|ㄱ-ㅎ|가-힣|a-z|A-Z|0-9]+\).AP)'
    removeINV = r'( and \([ -!:*|ㄱ-ㅎ|가-힣|a-z|A-Z|0-9]+\).INV)'
    removeGroup = r'|'.join((removeDate,removeAP,removeINV)) 

    def convert_symbols(v):
        result = v
        if v.startswith("-") or ' or -' in v:
            result = v.replace("-", "!")
        # convert nagative not to !
        if v.startswith("not ") or ' or not ' in v:
            result = v.replace("not ", "!")
        # convert wildcard * to :*
        if v.endswith("*") or '*' in v:
            result = v.replace("*", ":*")
        return result

    def re_sub(reg, to, val):
        return re.sub(reg, to, val, flags=re.IGNORECASE)

    def removeOuterParentheses(S: str) -> str:
        stack = []
        res = ''
        for i in range(len(S)):
            stack.append(S[i])
            if stack.count('(') == stack.count(')'):
                res+=''.join(stack[1:-1])
                stack = []
        if res == '':
            return S               
        return res

    def remove_tail(result, tail):
        if result.endswith(result):
            tail = -len(tail)
            result = result[:tail]
        return result

    def keepParantheses(v):
        pattern = re.compile("\||\<[\d+|-]\>")
        return re.search(pattern, v)

    if mode == 'terms':
        keyword = re_sub(removeGroup, r"", keyword)
    elif mode == 'person':
        keyword = re_sub(removeDate, r"", keyword)

    existOr = True if ' or ' in keyword else False
    # strKeyword = "(" if existOr else ''
    strKeyword = ''
    
    for v in re.split(" and ", keyword, flags=re.IGNORECASE):
        v = convert_symbols(v)
        v = re_sub(' or ', r"|", v)

        if '|' in v:
            v = removeOuterParentheses(v)
            strOr = '('
            for _v in re.split("\|", v):
                print(_v)
                _v = re_sub(adjZeroGroup, r"<->", _v)
                _v = re_sub(adjHaveNumberExecptZero, r"<\1>", _v)
                _v = re_sub(adjZeroGroup, r"<->", _v)
                _v = re_sub(adjHaveNumberExecptZero, r"<\1>", _v)
                _v = re_sub(nearHaveNumberExecptZero, r"(\1<\2>\3|\3<\2>\1)", _v)
                _v = re_sub(nearHaveOnlyZero, r"(\1<->\2|\2<->\1)", _v)
                _v = re_sub(nearOnly, r"(\1<->\2|\2<->\1)", _v)
                _v = re_sub(adjSpace, r"\1<->\2", _v)
                print(_v)
                _v = removeOuterParentheses(_v) if not keepParantheses(_v) else _v
                _v = re_sub(' ', r"&", _v)
                strOr += ("".join(str(_v)) + "|")
            strOr = remove_tail(strOr,"|")
            strKeyword += ("".join(str(strOr)) + ")&")
        else:
            v = re_sub(nearHaveNumberExecptZero, r"(\1<\2>\3|\3<\2>\1)", v)
            v = re_sub(nearHaveOnlyZero, r"(\1<->\2|\2<->\1)", v)
            v = re_sub(nearOnly, r"(\1<->\2|\2<->\1)", v)                            
    
            v = re_sub(adjSpace, r"\1<->\2", v)
            v = removeOuterParentheses(v) if not keepParantheses(v) else v
            v = re_sub(' ', r"&", v)
            # strKeyword += v + ")" if existOr else v
            # strKeyword += v
            strKeyword += ("".join(str(v)) + "&")
            # print(strKeyword)

    strKeyword = remove_tail(strKeyword,"&")
       

    if not strKeyword:
        return None

    result = f'"{fieldName}" @@ to_tsquery(\'{strKeyword}\')'
    # result = strKeyword
    return result
       



print(tsquery_keywords(keyword,'search'))

(예방) and (치료) and (진단) and (진단 near 키트 or 백신 조성물)
진단 near 키트
(진단<->키트|키트<->진단)
백신 조성물
백신 조성물
"search" @@ to_tsquery('예방&치료&진단&((진단<->키트|키트<->진단)|백신&조성물)')
